## 1. 분포가설과 분산표현
- 희소 표현(Sparse Reresentation): 벡터의 특정 차원에 단어 혹은 의미를 직접 매핑하는 방식

### 1-1. 단어의 분산 표현(Distributed Representation)
모든 단어를 고정 차원(예를 들어 256차원)의 벡터로 표현하는 것입니다. 그러나 어떤 차원이 특정한 의미를 가진다고 가정하지 않을 것입니다. 단지 이런 가정을 할 것입니다. [유사한 맥락에서 나타나는 단어는 그 의미도 비슷하다] ㅣ라는 것입니다. 이것을 분포 가설(distribution hypothesis)이라고 합니다.

- 맥락: 단순하게는 단어 좌우에 출현하는 다른 단어들
- 분산 표현을 사용하면 희소 표현과는 다르게 단어간의 유사도를 계산으로 구할 수 있다는 장점이 있다.
- Embedding 레이어는 바로 단어의 분산 표현을 구현하기 위한 레이어이다.
- "단어 n개 쓸거야~ k차원으로 표현해줘~" 라고 전달하면 컴퓨터가 알아서 [ n x k ] 형태의 분산 표현 사전을 만듭니다. 이것이 곧 Weight이고 파라미터이다.

### 1-2. Embedding 레이어
- 컴퓨터용 단어 사전
- 단어를 더 깊게 표현하고 싶은 경우 **Embedding 사이즈** 조정
- Weight는 자연스럽게 (단어의 갯수, Embedding 사이즈)로 정의된다.
- Embedding 레이어는 입력으로 들어온 단어를 분산 표현으로 연결해 주는 역할을 하는데 그것이 Weight에서 특정 행을 읽어오는 것과 같아 이 레이어를 룩업 테이블(Lookup Table)이라 한다.
- 원-핫 벡터로 표현된 서로 다른 두 단어가 항상 직교한다는 것은 모든 단어가 단어 간 관계를 전혀 반영하지 못한 채 각각 독립적인 상태로 존재하게 된다는 것을 의미한다.

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


In [2]:
#  생성된 원-핫 벡터를 Linear 레이어에 넣기
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[ 0.33984768 -0.40402743]
 [ 0.6891742   0.35254264]
 [-0.40114146  0.26264942]
 [-0.5351083  -0.6836436 ]
 [-0.16552454 -0.5277506 ]
 [ 0.40893245  0.2844603 ]
 [ 0.3679129  -0.01373708]
 [ 0.58081627  0.20242709]]

One-Hot Linear Result
[[ 0.33984768 -0.40402743]
 [ 0.33984768 -0.40402743]
 [ 0.33984768 -0.40402743]
 [ 0.33984768 -0.40402743]
 [ 0.6891742   0.35254264]
 [-0.40114146  0.26264942]
 [-0.5351083  -0.6836436 ]
 [-0.16552454 -0.5277506 ]
 [-0.16552454 -0.5277506 ]
 [-0.16552454 -0.5277506 ]]


원-핫 인코딩을 위한 단어 사전을 구축하고 단어를 사전의 인덱스로 변환만 해주면 Embedding 레이어를 완벽하게 사용할 수 있다!

In [3]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


#### 주의사항
- 딥러닝은 미분을 기반으로 동작하는 반면, 임베딩 레이어는 단어를 대응시켜 줄 뿐이기 때문에 미분이 불가능하다. 
- 따라서 신경망 설계를 할 때, **어떤 연산 결과를 임베딩 레이어에 연결시키는 것은 불가능하다.**
- 임베딩 레이어는 **입력에 직접 연결되게 사용**해야 한다.

## 2.  순차적인 데이터, Recurrent 레이어 (1) RNN
- 문장이나 영상, 음성 등의 데이터는 한 장의 이미지 데이터와는 사뭇 다른 특성을 가집니다. 바로 순차적인(Sequential) 특성이다.
- "시간"의 개념으로 이해한다.
- 데이터의 나열 사이에 연관성이 없다고 해서 순차적인 데이터가 아니라고 할 수는 없다. [1, 2, 3, 오리, baby, 0.7] 라는 데이터도 요소들 간의 연관성이 없지만 시퀀스 데이터라고 칭한다.
- 인공지능이 예측을 하기 위해선 요소 간의 연관성이 있어야만 합니다. 따라서 딥러닝에서 말하는 시퀀스 데이터는 **순차적인 특성**을 필수로 갖는다고 할 수 있다.

### Recurrent Neural Network 또는 Recurrent 레이어(RNN)
RNN의 입력으로 들어가는 모든 단어만큼 Weight를 만드는 게 아님에 유의합니다. (입력의 차원, 출력의 차원)에 해당하는 단 하나의 Weight를 순차적으로 업데이트하는 것이 RNN입니다. 그렇다 보니 한 문장을 읽고 처리하는 데에도 여러 번의 연산이 필요해 다른 레이어에 비해 느리다는 단점이 있어요.
- 문제점: 첫 입력의 정보가 마지막 입력에 다다라서는 거의 희석된다는 문제가 발생한다. 이를 **기울기 소실(Vanishing Gradient) 문제**라 한다.

In [4]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


어떤 문장이 긍정인지 부정인지 나누기 위해서라면 문장을 모두 읽은 후, 최종 Step의 Output만 확인해도 판단이 가능합니다. 하지만 문장을 생성하는 경우라면 이전 단어를 입력으로 받아 생성된 모든 다음 단어, 즉 모든 Step에 대한 Output이 필요하죠. 그것은 위 코드에서 tf.keras.layers.SimpleRNN 레이어의 return_sequences 인자를 조절함으로써 조절할 수 있다.

In [5]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


## 3. LSTM

- LSTM은 Long Short-Term Memory의 약어로 기울기 소실 문제를 해결하기 위해 고안된 RNN 레이어
- 기본적인 바닐라(Simple) RNN보다 4배나 큰 Weight를 가지고 있다.
- 4배 깊은 RNN이라고 표현하기 보다, 4종류의 서로 다른 Weight를 가진 RNN이라고 이해할 수 있다.

### LSTM의 3개의 Gate Layer
Forget Gate Layer : cell state의 기존 정보를 얼마나 잊어버릴지를 결정하는 gate

Input Gate Layer  : 새롭게 만들어진 cell state를 기존 cell state에 얼마나 반영할지를 결정하는 gate

Output Gate Layer : 새롭게 만들어진 cell state를 새로운 hidden state에 얼마나 반영할지를 결정하는 gate

###  LSTM의 변형 모델에 소개된 "엿보기 구멍(peephole connection)"
기존에는 LSTM의 Gate들이 input와 hidden state만을 참조하여 값을 경정하였는데, peephole을 통해  cell state를 함께 쳐다보게 만드는 것

### GRU의 특징
LSTM의 Forget Gate와 Inpurt Gate를 Update Gate로 합쳤다. 그리고 Cell State와 Hidden State를 합쳤다.

 Gated Recurrent Unit(GRU)도 tf.keras.layers.GRU() 로 선언해서 사용할 수 있다.
 
 LSTM에 비해 GRU가 학습할 가중치(Weight)가 더 적다. (LSTM의 3/4)
 
 LSTM은 GRU에 비해 Weight가 많기 때문에 충분한 데이터가 있는 상황에 적합하고, 반대로 GRU는 적은 데이터에도 웬만한 학습 성능을 보여준다.

### 양방향(Bidirectional) RNN
순방향 또는 역방향의 해결하기 위해 제안된 것이 양방향(Bidirectional) RNN 이다. 말이 조금 어렵지만, 그저 진행 방향이 반대인 RNN을 2개 겹쳐놓은 형태이다.

tf.keras.layers.Bidirectional()

문장 분석이나 생성보다는 주로 기계번역 같은 태스크에 유리하다.

그러나 문장을 번역하려면 일단은 번역해야 할 문장 전체를 끝까지 분석한 후 번역을 시도하는 것이 훨씬 유리합니다. 그래서 자연어처리를 계속하면서 알게 되겠지만, 번역기를 만들때 양방향(Bidirectional) RNN 계열의 네트워크, 혹은 동일한 효과를 내는 Transformer 네트워크를 주로 사용하게 될 것입니다.



In [6]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


Bidirectional RNN은 순방향 Weight와 역방향 Weight를 각각 정의하므로 우리가 앞에서 배운 RNN의 2배 크기 Weight가 정의됩니다. units를 64로 정의해 줬고, 입력은 Embedding을 포함하여 (1, 5, 100), 그리고 양방향에 대한 Weight를 거쳐 나올 테니 출력은 (1, 5, 128) 